In [ ]:
import plotly
import json
import requests
import pandas as pd
import geopandas as gpd
import plotly.graph_objects as go
from urllib.request import urlopen
from nowcasting_forecast.database.models import Forecast, ManyForecasts

URL = 'XXX.com'

In [ ]:
# Get GSP boundaries

r = requests.get(URL+'/v0/forecasts/GB/pv/gsp_boundaries/')
d = json.loads(r.json())
boundaries = gpd.GeoDataFrame.from_features(d["features"])


In [ ]:
# get forecasts from API
fileobj = urlopen(URL+'/v0/forecasts/GB/pv/gsp/') 
d = json.loads(fileobj.read())
forecasts = ManyForecasts(**d)


In [ ]:
# format predictions
time = forecasts.forecasts[1].forecast_values[0].target_time
predictions = {f.location.gsp_id:f.forecast_values[0].expected_power_generation_megawatts for f in forecasts.forecasts}
predictions_df = pd.DataFrame(list(predictions.items()),columns=['gsp_id','value'])

boundaries_and_results = boundaries.join(predictions_df, on=['gsp_id'], rsuffix='_r')


In [ ]:
# plot
boundaries_and_results = boundaries_and_results[~boundaries_and_results.RegionID.isna()]


# make shape dict for plotting
shapes_dict = json.loads(boundaries_and_results.to_json())

# plot it
fig = go.Figure()
fig.add_trace(
    go.Choroplethmapbox(
        geojson=shapes_dict, 
        locations=boundaries_and_results.index, 
        z=boundaries_and_results.value, colorscale="Viridis",
        zmax=1,
        zmin=0
    )
)

fig.update_layout(mapbox_style="carto-positron", mapbox_zoom=4, mapbox_center={"lat": 55, "lon": 0})
fig.update_layout(margin={"r": 0, "t": 30, "l": 0, "b": 30})
fig.update_layout(title=time.isoformat())
fig.show()
